# 1. Install Dependencies and Setup

In [1]:
#!pip install tensorflow tensorflow opencv-python matplotlib

In [2]:
#!pip list

In [3]:
import tensorflow as tf
from tensorflow import keras
import os

In [4]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
tf.config.list_physical_devices('GPU')

[]

# 2. Remove dodgy images

In [ ]:
import cv2
import imghdr

In [ ]:
data_dir = 'data_cars'

In [ ]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [ ]:
num_obs = [0,0]
class_idx = -1
for image_class in os.listdir(data_dir):
    class_idx = class_idx + 1
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
            else:
                num_obs[class_idx] = num_obs[class_idx] + 1
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

# 3. Load Data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# build data from directories of images
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory
data = tf.keras.utils.image_dataset_from_directory(data_dir)

In [ ]:
class_names = data.class_names
print(class_names)

In [ ]:
plt.bar([class_names[0],class_names[1]], num_obs);

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][4:8]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][4+idx])

# 4. Scale Data

In [ ]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
data.as_numpy_iterator().next()

# 5. Split Data

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [ ]:
train_size

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# 6. Build Deep Learning Model

In [ ]:
n_epochs = 8

In [ ]:
train

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile('adam',
              loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy',
                       tf.keras.metrics.Precision(),
                       tf.keras.metrics.Recall()])

In [ ]:
model.summary()

# 7. Train

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=n_epochs, validation_data=val, callbacks=[tensorboard_callback])

# 8. Plot Performance

In [ ]:
 def plot_metrics(history, class_names):
    # percision = fraction of predicted 'trues' that are correct
    # recall = fraction of 'trues' that were found
    fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(12,6))
    ls = ['-','--','.']
    j = 0
    for hist in history:
        metric_names = list(hist.history.keys())
        for idx in range(2):
            ax[0,idx].plot(hist.history[metric_names[idx]], color='teal', linestyle=ls[j], label=metric_names[idx])
            ax[0,idx].plot(hist.history['val_'+metric_names[idx]], color='orange', linestyle=ls[j], label='val_'+metric_names[idx])
            ax[0,idx].set_title(metric_names[idx], fontsize=10)
            ax[0,idx].legend()
        for idx in range(2):
            ax[1,idx].plot(hist.history[metric_names[idx+2]], color='teal', linestyle=ls[j], label=metric_names[idx+2])
            ax[1,idx].plot(hist.history['val_'+metric_names[idx+2]], color='orange', linestyle=ls[j], label='val_'+metric_names[idx+2])
            ax[1,idx].set_title(metric_names[idx+2], fontsize=10)
            ax[1,idx].legend()
        j = j + 1
    fig.suptitle('Evaluation Metrics - '+class_names[0]+' vs '+class_names[1], fontsize=20)

In [ ]:
plot_metrics([hist], class_names)

In [ ]:
# save the history from validating this model for future comparison
base_model_hist = hist

# 9. Evaluate

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

# 10. Test

In [ ]:
import cv2

In [ ]:
img = 255-cv2.imread(data_dir+'/minivan/2015-toyota-sienna-front-right.jpg')
#img = cv2.cvtColor(255-img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
print(yhat)

In [ ]:
if yhat > 0.5: 
    print(f'Predicted class is Minivan')
else:
    print(f'Predicted class is Ferrari')

# 11. Save the Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
mode_fname = os.path.join('models','imageclassifier.h5')
model.save(mode_fname)

In [ ]:
new_model = load_model(mode_fname)

In [ ]:
new_model.predict(np.expand_dims(resize/255, 0))

# Regularization

In [ ]:
model = Sequential()
# see: https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal
# see: https://www.tensorflow.org/api_docs/python/tf/keras/regularizers/L2
model.add(Conv2D(16, (3,3), 1, activation='elu', input_shape=(256,256,3),
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='elu',
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='elu',
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='elu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer="nadam",
              loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy',
                       tf.keras.metrics.Precision(),
                       tf.keras.metrics.Recall()])
# loss="sparse_categorical_crossentropy"
# or l1(0.1) for ℓ1 regularization with a factor or 0.1
# or l1_l2(0.1, 0.01) for both ℓ1 and ℓ2 regularization, with factors 0.1 and 0.01 respectively

In [ ]:
hist = model.fit(train, epochs=n_epochs, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
plot_metrics([base_model_hist, hist], class_names)

In [ ]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

# Test the regularized model

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)
print(pre.result(), re.result(), acc.result())